In [1]:
import matplotlib
matplotlib.use('TKAgg')
%load_ext autoreload
%autoreload 2
import os 
import datetime
import glob 
import numpy as np
import matplotlib.pyplot as plt
from pianoq.simulations.mplc_sim.mplc_sim import MPLCSim
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.singles_scan import signal_scan, idler_scan, get_signal_scanner, get_idler_scanner
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots, get_imaging_masks
from pianoq_results.scan_result import ScanResult
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCSimResult
from pianoq.lab.mplc.mplc_device import MPLCDevice
from pianoq.lab.photon_scan import PhotonScanner
from pianoq.misc.misc import run_in_thread, run_in_thread_simple
from pianoq.misc.mplt import mimshow, mplot
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_diffuser_masks
from pianoq.lab.mplc.discrete_photon_scanner import DiscretePhotonScanner
import time 
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.find_discreet_phases import PhaseFinder

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_24_zernike'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])
U_no = 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Motors
backlash = 0
wait_after_move = 0.3
from pianoq.lab.mplc.consts import thorlabs_x_serial, thorlabs_y_serial
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.zaber_motor import ZaberMotors
zaber_ms = ZaberMotors(backlash=backlash, wait_after_move=wait_after_move)
mxs = zaber_ms.motors[1]
mys = zaber_ms.motors[0]
print('Got Zaber motors')

mxi = ThorlabsKcubeDC(thorlabs_x_serial, backlash=backlash, wait_after_move=wait_after_move)
myi = ThorlabsKcubeStepper(thorlabs_y_serial, backlash=backlash, wait_after_move=wait_after_move)
print('Got Thorlabs motors')

# MPLC
mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')

# Timetagger
from pianoq.lab.time_tagger import QPTimeTagger
from pianoq.lab.mplc.consts import TIMETAGGER_DELAYS, TIMETAGGER_COIN_WINDOW
tt = QPTimeTagger(integration_time=1, remote=True,
                  single_channel_delays=TIMETAGGER_DELAYS, coin_window=TIMETAGGER_COIN_WINDOW)
print('Got Time tagger')

Got Zaber motors
Got Thorlabs motors
Got MPLC
Got Time tagger


# Optimization
## prepare

In [5]:
# masks 
masks_path = glob.glob(rf'{dir_path}\U{U_no}U*.masks')[0]
msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
mplc.load_masks(masks, linear_tilts=True)

# locs 
locs_idl_path = glob.glob(fr'{dir_path}\*idl*.locs')[0]
locs_sig_path = glob.glob(fr'{dir_path}\*sig*.locs')[0]
locs_idl = np.load(locs_idl_path)['locs']
locs_sig = np.load(locs_sig_path)['locs']

# i,j of corr matrix that is supposed to be strong 
i = 2
j = 2
mxi.move_absolute(locs_idl[i, 0])
myi.move_absolute(locs_idl[i, 1])
mxs.move_absolute(locs_sig[j, 0])
mys.move_absolute(locs_sig[j, 1])        
mplc.restore_location()
time.sleep(1)

## optimize

In [16]:
from aotools.functions import phaseFromZernikes
Z_MASK_SIZE = 360
z_slice = np.index_exp[:, 110:250]

# find phases
mplc.load_masks(masks, linear_tilts=True) 
orig_masks = mplc.masks.copy()

iters = 3
N_zernike = 14
N_mags = 10
magnitude_range = np.linspace(-3, 3, N_mags)
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
integration_time = 10
tt.set_integration_time(integration_time)

all_costs = []
v_zernike = np.zeros(N_zernike)
best_z_vals = np.zeros(N_zernike)

for iter in range(iters):
    print(f'{iter=}')
    for z_coef in range(N_zernike):
        print(f'{z_coef=}')
        now_costs = np.zeros(len(magnitude_range))
        for i, magnitude in enumerate(magnitude_range):
            vals = best_z_vals.copy()
            vals[z_coef] = magnitude
            z_mask = phaseFromZernikes(vals, Z_MASK_SIZE)
            new_masks = orig_masks.copy()
            new_masks[0] *= np.exp(1j*z_mask[z_slice])
            mplc.load_masks(new_masks, linear_tilts=True)
            s1, s2, c = tt.read_interesting()
            cost = c - 2*tt.coin_window*s1*s2
            print(f'\t{i}: {cost=:.3f}')
            now_costs[i] = cost
        all_costs.append(now_costs)
        best_ind = np.argmax(now_costs)
        best_z_vals[z_coef] = magnitude_range[best_ind]
    print(best_z_vals)

iter=0
z_coef=0
	0: cost=0.9371720592400001
	1: cost=1.2312467708800001
	2: cost=1.8255185426
	3: cost=1.70790281872
	4: cost=1.7943334391199999
	5: cost=1.69354307296
	6: cost=1.6848694480000002
	7: cost=2.9852170790000003
	8: cost=2.08731376436
	9: cost=1.49385155548
	10: cost=2.39479229912
	11: cost=2.6995353473600003
	12: cost=2.29772696092
	13: cost=1.69887888684
	14: cost=1.6972643062000001
	15: cost=1.607659458
	16: cost=0.43088955516000005
	17: cost=1.3271500194799999
	18: cost=0.931174688
	19: cost=1.04035099216
z_coef=1
	0: cost=0.80599237912
	1: cost=0.9087748198400001
	2: cost=1.7072254475200002
	3: cost=1.7942315408
	4: cost=1.402516087
	5: cost=1.7883398942
	6: cost=2.0887285937200004
	7: cost=1.58810859648
	8: cost=2.28447784096
	9: cost=1.58386245872
	10: cost=2.0875327834800004
	11: cost=1.2859292544
	12: cost=1.6848468688000002
	13: cost=1.58640815972
	14: cost=1.1882384508800001
	15: cost=2.9904350594
	16: cost=1.2898382944
	17: cost=0.8919750152000001
	18: cost=2.09

In [19]:
mimshow(np.array(all_costs))

(<Figure size 640x480 with 2 Axes>, <Axes: >)